In [1]:
import torch
from torch.utils.data import DataLoader

In [2]:
device = torch.device('cuda:0')

data_lst = ['Set5', 'Set14', 'BSD100']

# **ABPN**

In [3]:
from Attention.ABPN.dataset import *
from Attention.ABPN.model import *
from Attention.ABPN.utils import *

In [4]:
mag = 4

ori = ABPN_v5(input_dim=3, dim=32)
ori.load_state_dict(torch.load('./Attention/ABPN/Models/ABPNv5_X4.pt'))
ori = ori.to(device)

modi = ABPNv5_LaUD(input_dim=3, dim=48)
modi.load_state_dict(torch.load('./Attention/ABPN/Models/ABPNv5_LaUD_X4.pt'))
modi = modi.to(device)

In [5]:
for testdata in data_lst:
    testset = test_dataset(root_path='./datasets/for_test', type=testdata,
                           is_resize=False, resize_h=None, resize_w=None, is_rcrop=False, crop_h=None, crop_w=None, scale=mag, 
                           is_rrot=False, rand_hori_flip=False, rand_vert_flip=False, grayscale=False, norm=False)
    testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)

    ori.eval()
    modi.eval()
    with torch.no_grad():
        ori_psnr_sum = 0 
        ori_ssim_sum = 0
        modi_psnr_sum = 0 
        modi_ssim_sum = 0
        for iteration, data in enumerate(testloader):
            hr_img, lr_img = data[0].to(device), data[1].to(device)

            sr_ori = ori(lr_img)
            sr_modi, det_modi = modi(lr_img)

            ori_psnr_sum += cal_psnr(hr_img, sr_ori, crop_border=mag, minmax='0_1', clamp=True, gray_scale=True, ver='YCrCb_BT601')
            ori_ssim_sum += cal_ssim(hr_img, sr_ori, crop_border=0, minmax='0_1', filter_size=11, filter_sigma=1.5, clamp=True, grayscale=True, ver='YCrCb_BT601')
            
            modi_psnr_sum += cal_psnr(hr_img, sr_modi, crop_border=mag, minmax='0_1', clamp=True, gray_scale=True, ver='YCrCb_BT601')
            modi_ssim_sum += cal_ssim(hr_img, sr_modi, crop_border=0, minmax='0_1', filter_size=11, filter_sigma=1.5, clamp=True, grayscale=True, ver='YCrCb_BT601')

        print('[For {} X{}, PSNR and SSIM]\n ABPN: {:.2f} dB and {:.4f}\n ABPN_LaUD: {:.2f} dB and {:.4f}\n'.format(testdata, mag, ori_psnr_sum/len(testloader), ori_ssim_sum/len(testloader),
                                                                                                 modi_psnr_sum/len(testloader), modi_ssim_sum/len(testloader)))            

[For Set5 X4, PSNR and SSIM]
 ABPN: 32.28 dB and 0.8955
 ABPN_LaUD: 32.60 dB and 0.8991

[For Set14 X4, PSNR and SSIM]
 ABPN: 28.67 dB and 0.7828
 ABPN_LaUD: 28.90 dB and 0.7882

[For BSD100 X4, PSNR and SSIM]
 ABPN: 27.61 dB and 0.7379
 ABPN_LaUD: 27.76 dB and 0.7433



# **DRLN**

In [6]:
from Attention.DRLN.dataset import *
from Attention.DRLN.model import *
from Attention.DRLN.utils import *

In [7]:
mag = 2

ori = DRLN(scale=mag)
ori.load_state_dict(torch.load('./Attention/DRLN/Models/DRLN_X2.pt'))
ori = ori.to(device)

modi = DRLN_LaUD(scale=mag)
modi.load_state_dict(torch.load('./Attention/DRLN/Models/DRLN_LaUD_X2.pt'))
modi = modi.to(device)

In [8]:
for testdata in data_lst:
    testset = test_dataset(root_path='./datasets/for_test', type=testdata,
                           is_resize=False, resize_h=None, resize_w=None, is_rcrop=False, crop_h=None, crop_w=None, scale=mag, 
                           is_rrot=False, rand_hori_flip=False, rand_vert_flip=False, grayscale=False, norm=False)
    testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)

    ori.eval()
    modi.eval()
    with torch.no_grad():
        ori_psnr_sum = 0 
        ori_ssim_sum = 0
        modi_psnr_sum = 0 
        modi_ssim_sum = 0
        for iteration, data in enumerate(testloader):
            hr_img, lr_img = data[0].to(device), data[1].to(device)

            sr_ori = ori(lr_img)
            sr_lst_modi, det_lst_modi = modi(lr_img)

            ori_psnr_sum += cal_psnr(hr_img, sr_ori, crop_border=mag, minmax='0_1', clamp=True, gray_scale=True, ver='YCrCb_BT601')
            ori_ssim_sum += cal_ssim(hr_img, sr_ori, crop_border=0, minmax='0_1', filter_size=11, filter_sigma=1.5, clamp=True, grayscale=True, ver='YCrCb_BT601')
            
            modi_psnr_sum += cal_psnr(hr_img, sr_lst_modi[-1], crop_border=mag, minmax='0_1', clamp=True, gray_scale=True, ver='YCrCb_BT601')
            modi_ssim_sum += cal_ssim(hr_img, sr_lst_modi[-1], crop_border=0, minmax='0_1', filter_size=11, filter_sigma=1.5, clamp=True, grayscale=True, ver='YCrCb_BT601')

        print('[For {} X{}, PSNR and SSIM]\n DRLN: {:.2f} dB and {:.4f}\n DRLN_LaUD: {:.2f} dB and {:.4f}\n'.format(testdata, mag, ori_psnr_sum/len(testloader), ori_ssim_sum/len(testloader),
                                                                                                 modi_psnr_sum/len(testloader), modi_ssim_sum/len(testloader)))            

[For Set5 X2, PSNR and SSIM]
 DRLN: 38.10 dB and 0.9610
 DRLN_LaUD: 38.35 dB and 0.9623

[For Set14 X2, PSNR and SSIM]
 DRLN: 33.75 dB and 0.9188
 DRLN_LaUD: 34.49 dB and 0.9241

[For BSD100 X2, PSNR and SSIM]
 DRLN: 32.24 dB and 0.9006
 DRLN_LaUD: 32.49 dB and 0.9039



# **HAN**

In [9]:
from Attention.HAN.dataset import *
from Attention.HAN.model import *
from Attention.HAN.utils import *

In [10]:
mag = 2

ori = HAN(reduction=16, scale=mag)
ori.load_state_dict(torch.load('./Attention/HAN/Models/HAN_X2.pt'))
ori = ori.to(device)

modi = HAN_LaUD(reduction=16, scale=mag)
modi.load_state_dict(torch.load('./Attention/HAN/Models/HAN_LaUD_X2.pt'))
modi = modi.to(device)

In [11]:
for testdata in data_lst:
    testset = test_dataset(root_path='./datasets/for_test', type=testdata,
                           is_resize=False, resize_h=None, resize_w=None, is_rcrop=False, crop_h=None, crop_w=None, scale=mag, 
                           is_rrot=False, rand_hori_flip=False, rand_vert_flip=False, grayscale=False)
    testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)

    ori.eval()
    modi.eval()
    with torch.no_grad():
        ori_psnr_sum = 0 
        ori_ssim_sum = 0
        modi_psnr_sum = 0 
        modi_ssim_sum = 0
        for iteration, data in enumerate(testloader):
            hr_img, lr_img = data[0].to(device), data[1].to(device)

            sr_ori = ori(lr_img)
            sr_modi, det_modi = modi(lr_img)

            ori_psnr_sum += cal_psnr(hr_img, sr_ori, crop_border=mag, minmax='0_255', clamp=True, gray_scale=True, ver='YCrCb_BT601')
            ori_ssim_sum += cal_ssim(hr_img, sr_ori, crop_border=0, minmax='0_255', filter_size=11, filter_sigma=1.5, clamp=True, grayscale=True, ver='YCrCb_BT601')
            
            modi_psnr_sum += cal_psnr(hr_img, sr_modi, crop_border=mag, minmax='0_255', clamp=True, gray_scale=True, ver='YCrCb_BT601')
            modi_ssim_sum += cal_ssim(hr_img, sr_modi, crop_border=0, minmax='0_255', filter_size=11, filter_sigma=1.5, clamp=True, grayscale=True, ver='YCrCb_BT601')

        print('[For {} X{}, PSNR and SSIM]\n HAN: {:.2f} dB and {:.4f}\n HAN_LaUD: {:.2f} dB and {:.4f}\n'.format(testdata, mag, ori_psnr_sum/len(testloader), ori_ssim_sum/len(testloader),
                                                                                                 modi_psnr_sum/len(testloader), modi_ssim_sum/len(testloader)))            

[For Set5 X2, PSNR and SSIM]
 HAN: 38.26 dB and 0.9616
 HAN_LaUD: 38.31 dB and 0.9617

[For Set14 X2, PSNR and SSIM]
 HAN: 34.11 dB and 0.9217
 HAN_LaUD: 34.15 dB and 0.9220

[For BSD100 X2, PSNR and SSIM]
 HAN: 32.39 dB and 0.9027
 HAN_LaUD: 32.41 dB and 0.9030

